In [1]:
import igraph as ig
import graph_importer as gim
import numpy as np
import polars as pl

In [2]:
G = gim.make_graph('concordant_networks/temponet_14_365.lgl')
S = ig.Graph.Read_GraphML('conc_tempo_14_in/0014.graphml')

In [3]:
# simple examine attrs of edges

edge_time = [(e.source_vertex['time'], e.target_vertex['time']) for e in G.es]
edge_time_df = pl.from_numpy(np.asanyarray(edge_time), schema=['from_time', 'to_time'])
edge_time_df.with_columns(pl.col('from_time').eq(pl.col('to_time')).alias('same_time')).group_by('same_time').count()

same_time,count
bool,u32
false,3936478
true,521059


In [4]:
edge_loc = [(e.source_vertex['loc'], e.target_vertex['loc']) for e in G.es]
edge_loc_df = pl.from_numpy(np.asanyarray(edge_loc), schema=['from_loc', 'to_loc'])
edge_loc_df.with_columns(pl.col('from_loc').eq(pl.col('to_loc')).alias('same_loc')).group_by('same_loc').count()

same_loc,count
bool,u32
true,772135
false,3685402


In [5]:
edge_attrs = [(e.source_vertex['loc'], 
               e.source_vertex['time'], 
               e.target_vertex['loc'],
               e.target_vertex['time']) 
              for e in G.es]

In [6]:
edge_attrs_df = pl.from_numpy(np.asanyarray(edge_attrs), schema=['from_loc', 'from_time', 'to_loc', 'to_time'])

In [11]:
edge_attrs_df = edge_attrs_df.with_columns(
    pl.col('from_loc').eq(pl.col('to_loc')).alias('same_loc'),
    pl.col('from_time').eq(pl.col('to_time')).alias('same_time'),
).with_columns(
    pl.col('same_loc').or_(pl.col('same_time')).alias('orthog_edge')
)

In [12]:
edge_attrs_df.group_by('orthog_edge').count()

orthog_edge,count
bool,u32
false,3213130
true,1244407
